<br>
@Author: Ayush Prajapati<br>
@Date: 11-09-2024<br>
@Last Modified by: Ayush Prajapati<br>
@Last Modified time: 11-09-2024<br>
@Title: S3 CRUD operations using Boto3 Library<br>
<br>

In [1]:
%pip install boto3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
import boto3
from dotenv import load_dotenv
from io import StringIO
import pandas as pd

In [4]:
load_dotenv()

True

## Configuring boto3 with s3

**Create a boto3 client**

In [5]:
s3 = boto3.client('s3')

In [6]:
def create_s3_bucket(bucket_name, region = 'us-west-2'):
    """
    Description:
        Creates a s3 bucket in AWS
    Parameters:
        bucket_name = name of bucket
        region = name of region(optional)
    Returns:
        None
    """
    try:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region,
            },
        )
        print(f'Bucket {bucket_name} created successfully.')
        
    except Exception as e:
        print(f"Error: {e}")

**Creating a bucket**

In [8]:
create_s3_bucket('ayush-boto-crud-bucket')

Bucket ayush-boto-crud-bucket created successfully.


**List all buckets**

In [9]:
response = s3.list_buckets()

# Output bucket names
for bucket in response['Buckets']:
    print(bucket['Name'])

ayush-boto-crud-bucket
ayush-demo-bucket1


## Python Crud Operations

### Create Operation

In [10]:
def upload_file_to_s3(bucket_name,file_name,s3_key):
    """
    Description:
        Upload a file to s3 bucket in AWS
    Parameters:
        bucket_name = name of bucket
        file_name = name of file
        s3_key = name of file in s3
    Returns:
        None
    """
    try:
        s3.upload_file(file_name, bucket_name, s3_key)
        print(f"File {file_name} uploaded successfully to {bucket_name}/{s3_key}")
    except Exception as e:
        print(f"Error uploading file: {e}")

In [11]:
def main():
    bucket_name = 'ayush-boto-crud-bucket'
    csv_file_name = 'demo_data.csv'
    s3_key = 'csv/demo_data.csv'

    upload_file_to_s3(bucket_name, csv_file_name, s3_key)


if __name__ == '__main__':
    main()

File demo_data.csv uploaded successfully to ayush-boto-crud-bucket/csv/demo_data.csv


## Read Operation

In [16]:
def download_file_from_s3(bucket_name,s3_key,file_name):
    """
    Description:
        Download a file from s3 bucket in AWS to local
    Parameters:
        bucket_name = name of bucket
        s3_key = name of file in s3
        file_name = local name of file
    Returns:
        None
    """
    try:
        s3.download_file(bucket_name, s3_key, file_name)
        print(f"File {s3_key} downloaded successfully from {bucket_name} to {file_name}")
    except Exception as e:
        print(f"Error downloading file: {e}")

In [19]:
def view_csv_from_s3(bucket_name, s3_key):
    """
    Description:
        View a csv file from s3 bucket in AWS to local
    Parameters:
        bucket_name = name of bucket
        s3_key = name of file in s3
    Returns:
        None
    """
    try:
        # Fetch the file from S3
        obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
        csv_content = obj['Body'].read().decode('utf-8')
        
        # Use StringIO to treat the string as a file
        csv_string_io = StringIO(csv_content)
        
        df = pd.read_csv(csv_string_io)
        print("CSV content:")
        print(df)
        
    except Exception as e:
        print(f"Error reading file from S3: {e}")


def main():
    bucket_name = 'ayush-boto-crud-bucket'
    s3_key = 'csv/demo_data.csv'

    # To directly view the csv without downloading
    view_csv_from_s3(bucket_name, s3_key)

    # To download the file
    print("\nDownloading file....")
    download_file_from_s3(bucket_name, s3_key, 'data_from_s3.csv')


if __name__ == '__main__':
    main()


CSV content:
   id     name  age           city
0   1    Alice   30       New York
1   2      Bob   25  San Francisco
2   3  Charlie   28    Los Angeles
3   4     Dave   35        Chicago
4   5      Eve   29        Houston
5   6    Frank   33        Seattle
6   7    Grace   32          Miami
7   8     Hank   27         Boston

File csv/demo_data.csv downloaded successfully from ayush-boto-crud-bucket to downloaded_data_from_s3.csv


## Update Operation

In [25]:
def update_file_in_s3(file_name, bucket_name,s3_key):
    """
    Description:
        Mdify file in s3 bucket in AWS 
    Parameters:
        file_name: local name of file
        bucket_name = name of bucket
        s3_key = name of file in s3
    Returns:
        None
    """
    # Modify the file locally before re-uploading
    with open(file_name, 'a') as file:
        file.write("9,Isaac,40,Denver\n")
    
    # Re-upload the updated file
    upload_file_to_s3(bucket_name, file_name, s3_key)
    print(f"File {file_name} updated and uploaded to {bucket_name}/{s3_key}")


def main():
    file_name = 'demo_data.csv'
    bucket_name = 'ayush-boto-crud-bucket'
    s3_key = 'csv/demo_data.csv'

    # updating file
    update_file_in_s3(file_name, bucket_name, s3_key)


if __name__ == '__main__':
    main()

File demo_data.csv uploaded successfully to ayush-boto-crud-bucket/csv/demo_data.csv
File demo_data.csv updated and uploaded to ayush-boto-crud-bucket/csv/demo_data.csv


## Delete Operation

In [26]:
def delete_file_from_s3(bucket_name, s3_key):
    """
    Description:
        Delete file in s3 bucket in AWS 
    Parameters:
        file_name: local name of file
        bucket_name = name of bucket
        s3_key = name of file in s3
    Returns:
        None
    """
    try:
        s3.delete_object(Bucket=bucket_name, Key=s3_key)
        print(f"File {s3_key} deleted successfully from {bucket_name}")
    except Exception as e:
        print(f"Error deleting file: {e}")


def main():
    bucket_name = 'ayush-boto-crud-bucket'
    s3_key = 'csv/demo_data.csv'

    # deleting file
    delete_file_from_s3(bucket_name, s3_key)


if __name__ == '__main__':
    main()

File csv/demo_data.csv deleted successfully from ayush-boto-crud-bucket


**Closing Connections**

In [27]:
s3.close()